# **Credit Risk Cleaning Data**
This notebook cleans and preprocesses all the data from the files of the "Encuestas Financieras de las Familias".

The process consist in merging the different tables using the most meaningful data and solving the gaps generated by the different structures of the data obtained in the 2017 surve realized by the spanish central bank.

# **Install required packages**

In [ ]:
!pip install numpy
!pip install plotly
!pip install requests
!pip install PyPdf
!pip install leap_ec
!pip install torch
!pip install scikit-learn

# **Imports**


In [278]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import torch
import sklearn
import os

# **URL of the dataset**
All the datasets are in the "eff" folder of my github repository

In [279]:
#2017
url_dataset_2017 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2017/5/otras_secciones_2017_imp5.csv'
url_dataset_seccion6_2017 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2017/5/seccion6_2017_imp5.csv'

#2014
url_dataset_2014 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2014/5/otras_secciones_2014_imp5.csv'
url_dataset_seccion6_2014 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2014/5/seccion6_2014_imp5.csv'

#2011
url_dataset_2011 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2011/5/otras_secciones_2011_imp5.csv'
url_dataset_seccion6_2011 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2011/5/seccion6_2011_imp5.csv'

#2008
url_dataset_2008 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2008/5/otras_secciones_2008_imp5.csv'
url_dataset_seccion6_2008 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2008/5/seccion6_2008_imp5.csv'

#2005
url_dataset_2005 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2005/5/otras_secciones_2005_imp5.csv'
url_dataset_seccion6_2005 = 'https://raw.githubusercontent.com/simoncraf/tfg/main/eff/2005/5/seccion6_2005_imp5.csv'

# **Legend**

# **Pipeline**

In [280]:
def pipeline(url, url_sec6,year,prev_year):
    
    df = pd.read_csv(url, sep = ';')
    
    
    cols = [year,prev_year,'pan_1','p1_2d_1','p1_4_1','p1_5_1','p2_1','p2_5','p2_8','p2_12_1','p2_12_2','p2_12_3','p2_12_4',
          'p2_13_1','p2_13_2','p2_13_3','p2_13_4','p2_17_1','p2_17_2','p2_17_3','p2_17_4','p2_18_1','p2_18_2',
          'p2_18_3','p2_18_4','p2_31','p2_32','p2_55_4','p2_61_4','p3_6_1','p3_6_2','p3_6_3','p3_6_4','p3_6_5',
          'p3_6_6','p3_6_7','p3_6_8','p3_10_1','p3_10_2','p3_10_3','p3_10_4','p3_10_5','p3_10_6','p3_10_7','p3_10_8',
          'p3_11_1','p3_11_2','p3_11_3','p3_11_4','p3_11_5','p3_11_6','p3_11_7','p3_11_8','p4_24','p4_25','p4_15',
          'p4_16','p4_28a','p4_3','p4_33','p4_35','p4_39','p4_40','p4_5','p4_7_1','p4_7_3','p4_10','p4_18','p4_27',
          'p5_1','p5_4_1','p5_4_2','p5_4_3','p5_4_4','p5_4_5','p5_4_6','p5_4_7','p5_4_8','p5_4_9','p5_4_10',
          'p5_5b_1','p5_5b_2','p5_5b_3','p5_5b_4','p5_5b_5','p5_5b_6','p5_5b_7','p5_5b_8','p5_5b_9','p5_5b_10',
          'p5_6_1','p5_6_2','p5_6_3','p5_6_4','p5_6_5','p5_6_6','p5_6_7','p5_6_8','p5_6_9','p5_6_10',
          'p5_7_1','p5_7_2','p5_7_3','p5_7_4','p5_7_5','p5_7_6','p5_7_7','p5_7_8','p5_7_9','p5_7_10',
          'p9_1','p9_6','p9_11','p9_15a']
    
    
    df = df[cols]
    
    #Columns for mortgages
    columns_loans = ['p2_12_1','p2_12_2','p2_12_3','p2_12_4','p2_13_1','p2_13_2','p2_13_3','p2_13_4',
                     'p2_17_1','p2_17_2','p2_17_3','p2_17_4','p2_18_1','p2_18_2','p2_18_3','p2_18_4']

    #Columns for rents
    columns_rents = 'p2_31'
    '''
    p2_1: Régimen de tenencia de la vivienda principal
    1 -> Alquiler
    2 -> Propiedad
    3 -> Cesión gratuita
    '''
    df.loc[df['p2_1'] != 1, columns_rents] = 0
    df.loc[df['p2_1'] != 2, ['p2_8','p2_5']] = 0 #Indica que tant el valor de la casa on viu com les hipoteques son 0
    df.loc[df['p2_1'] != 2, columns_loans] = 0

    '''
    p2_8 -> Indica si te alguna hipoteca
    '''
    df.loc[df['p2_8'] == 2, columns_loans] = 0

    #Main Residence
    #Amounts outstanding to be repaid from all loans for the main home
    df['loans_tot_rest'] = df[['p2_12_1','p2_12_2','p2_12_3','p2_12_4']].sum(axis = 1)
    df = df.drop(columns=['p2_12_1','p2_12_2','p2_12_3','p2_12_4'])

    #Average annual interest rate on loans
    df['avg_loans_int'] = df[['p2_13_1','p2_13_2','p2_13_3','p2_13_4']].agg('mean',axis = 1)
    df = df.drop(columns=['p2_13_1','p2_13_2','p2_13_3','p2_13_4'])

    #Average outstanding term of loans
    df['avg_time_loans_rest'] = df[['p2_17_1','p2_17_2','p2_17_3','p2_17_4']].agg('mean',axis = 1)
    df = df.drop(columns=['p2_17_1','p2_17_2','p2_17_3','p2_17_4'])

    #Total monthly amount allocated to loans for the first home
    df['loans_tot_monthly'] = df[['p2_18_1','p2_18_2','p2_18_3','p2_18_4']].sum(axis = 1)
    df = df.drop(columns=['p2_18_1','p2_18_2','p2_18_3','p2_18_4'])

    df.loc[df['loans_tot_rest'] == 0.0, ['avg_loans_int','avg_time_loans_rest','loans_tot_monthly']] = 0

    df[['loans_tot_rest','avg_loans_int','avg_time_loans_rest','loans_tot_monthly']]
    
    #OTHER LOANS
    #Total amount of the other loans
    df['other_loans_tot_rest'] = df[['p3_6_1','p3_6_2','p3_6_3','p3_6_4','p3_6_5','p3_6_6','p3_6_7','p3_6_8']].sum(axis = 1)
    df = df.drop(columns=['p3_6_1','p3_6_2','p3_6_3','p3_6_4','p3_6_5','p3_6_6','p3_6_7','p3_6_8'])

    #Average outstanding term of other loans
    df['other_loans_avg_time_rest'] = df[['p3_10_1','p3_10_2','p3_10_3','p3_10_4','p3_10_5','p3_10_6','p3_10_7','p3_10_8']].agg('mean',axis = 1)
    df = df.drop(columns=['p3_10_1','p3_10_2','p3_10_3','p3_10_4','p3_10_5','p3_10_6','p3_10_7','p3_10_8'])

    #Total monthly amount allocated to other loans
    df['other_loans_tot_monthly'] = df[['p3_11_1','p3_11_2','p3_11_3','p3_11_4','p3_11_5','p3_11_6','p3_11_7','p3_11_8']].sum(axis = 1)
    df = df.drop(columns=['p3_11_1','p3_11_2','p3_11_3','p3_11_4','p3_11_5','p3_11_6','p3_11_7','p3_11_8'])

    df.loc[df['other_loans_tot_rest'] == 0, ['other_loans_avg_time_rest','other_loans_tot_monthly']] = 0

    df[['other_loans_tot_rest','other_loans_tot_monthly','other_loans_avg_time_rest']]
    
    columns_retirement = ['p5_4_1','p5_4_2','p5_4_3','p5_4_4','p5_4_5','p5_4_6','p5_4_7','p5_4_8','p5_4_9','p5_4_10',
                     'p5_6_1','p5_6_2','p5_6_3','p5_6_4','p5_6_5','p5_6_6','p5_6_7','p5_6_8','p5_6_9','p5_6_10',
                      'p5_7_1','p5_7_2','p5_7_3','p5_7_4','p5_7_5','p5_7_6','p5_7_7','p5_7_8','p5_7_9','p5_7_10']
    columns_age_retirement = ['p5_4_1','p5_4_2','p5_4_3','p5_4_4','p5_4_5','p5_4_6','p5_4_7','p5_4_8','p5_4_9','p5_4_10']
    columns_retirement_plans = ['p5_5b_1','p5_5b_2','p5_5b_3','p5_5b_4','p5_5b_5','p5_5b_6','p5_5b_7','p5_5b_8','p5_5b_9','p5_5b_10']
    columns_annual_ammount = ['p5_6_1','p5_6_2','p5_6_3','p5_6_4','p5_6_5','p5_6_6','p5_6_7','p5_6_8','p5_6_9','p5_6_10']
    columns_plan_value = ['p5_7_1','p5_7_2','p5_7_3','p5_7_4','p5_7_5','p5_7_6','p5_7_7','p5_7_8','p5_7_9','p5_7_10']

    '''
    p5_1 -> Has retirement plan? 
    1 -> Yes
    2 -> No
    '''

    df.loc[df['p5_1'] == 2, columns_retirement] = 0

    '''
    At what age do you intend to collect your pension plan?
    -1 -> No answer/ Do not know
    -3 -> Do not want to collect it since they prefer to leave it as an inheritance.
    '''
    for age in columns_age_retirement:
        df.loc[df[age] == -3, age] = 100
        df.loc[df[age] == -1, age] = np.NaN
    
    #Delete pension plans that do not belong to the main member of the survey
    '''
    p5_5b_X -> Miembro al que pertenece el plan de pensiones X
    p5_6_X -> Importe anual al plan de pensiones X
    p5_7_X -> Valor actualizado del plan
    '''

    for idx, plan in enumerate(columns_retirement_plans):
        df.loc[df[plan] != 1, columns_age_retirement[idx]] = np.NaN
        df.loc[df[plan] != 1, columns_annual_ammount[idx]] = np.NaN
        df.loc[df[plan] != 1, columns_plan_value[idx]] = np.NaN
        
    
    #PENSION PLANS: Putting values together

    #Average age at which you expect to receive the pension plan X
    df['avg_age_retirement_plans'] = df[['p5_4_1','p5_4_2','p5_4_3','p5_4_4','p5_4_5','p5_4_6','p5_4_7','p5_4_8','p5_4_9','p5_4_10']].agg('mean',axis = 1)
    df = df.drop(columns=['p5_4_1','p5_4_2','p5_4_3','p5_4_4','p5_4_5','p5_4_6','p5_4_7','p5_4_8','p5_4_9','p5_4_10'])

    #p5_6_X Amount of the annual contribution to the pension plan X
    df['tot_amm_year_retirement_plans'] = df[['p5_6_1','p5_6_2','p5_6_3','p5_6_4','p5_6_5','p5_6_6','p5_6_7','p5_6_8','p5_6_9','p5_6_10']].agg('mean',axis = 1)
    df = df.drop(columns=['p5_6_1','p5_6_2','p5_6_3','p5_6_4','p5_6_5','p5_6_6','p5_6_7','p5_6_8','p5_6_9','p5_6_10'])

    #p5_7_X -> Updated value of assets in the pension plan X
    df['tot_value_retirement_plans'] = df[['p5_7_1','p5_7_2','p5_7_3','p5_7_4','p5_7_5','p5_7_6','p5_7_7','p5_7_8','p5_7_9','p5_7_10']].agg('mean',axis = 1)
    df = df.drop(columns=['p5_7_1','p5_7_2','p5_7_3','p5_7_4','p5_7_5','p5_7_6','p5_7_7','p5_7_8','p5_7_9','p5_7_10'])

    public_inv_columns = ['p4_15','p4_16']
    no_pub_inv_columns = ['p4_24','p4_25']
    investment_funds = 'p4_28a'
    fix_rent = 'p4_35'
    ann_income_oth_inv = 'p4_40'
    saving_acc = 'p4_7_1'
    pay_acc = 'p4_7_3'


    df.loc[df['p4_10'] == 2, public_inv_columns] = 0
    df.loc[df['p4_18'] == 2, no_pub_inv_columns] = 0
    df.loc[df['p4_27'] == 2, investment_funds] = 0
    df.loc[df['p4_33'] == 2, fix_rent] = 0
    df.loc[df['p4_39'] == 2, ann_income_oth_inv] = 0
    df.loc[df['p4_3'] == 2, saving_acc] = 0
    df.loc[df['p4_5'] == 2, pay_acc] = 0

    investments_columns = ['p4_15','p4_16','p4_24','p4_25','p4_28a','p4_35','p4_40','p4_7_1','p4_7_3']
    
    #REVIEW THIS PART######################################
    
    '''
    p4_27 -> Has investments in investment funds
    p4_28a -> Value of those investments in investment funds
    '''
    #median_inv_funds = df['p4_28a'].loc[(df['p4_27'] == 1) and (df['p4_28a'].notna() == True)].median()
    #df.loc[(df['p4_27'] == 1) & (df['p4_28a'].isna()), 'p4_28a'] = median_inv_funds

    
    #######################################################
    
    #Seccion 6 2017

    df_6 = pd.read_csv(url_sec6, sep = ';')
    
    cols_sec6 = [year,'p6_14_1_1','p6_14_1_2','p6_14_1_3','p6_102_1_1','p6_102_1_2','p6_102_1_3','p6_49_1_1',
                'p6_49_1_2','p6_49_1_3','p6_49_1_4']
    
    df_6 =   df_6[cols_sec6]
    #p6_14_1_X -> Gross monthly salary
    df_6['tot_income_month'] = df_6[['p6_14_1_1','p6_14_1_2','p6_14_1_3','p6_102_1_1','p6_102_1_2','p6_102_1_3','p6_49_1_1','p6_49_1_2','p6_49_1_3','p6_49_1_4']].sum(axis = 1)
    df_6 = df_6.drop(columns=['p6_14_1_1','p6_14_1_2','p6_14_1_3','p6_102_1_1','p6_102_1_2','p6_102_1_3','p6_49_1_1','p6_49_1_2','p6_49_1_3','p6_49_1_4'])
    
    df = df.join(df_6.set_index([year]), on=[year],)
    
    df = df.rename(columns={"p1_1_1":"gender",'p1_2d_1':'age','p1_4_1':'married','p1_5_1':'education',
                       'p2_5':'value_main_house','p2_31':'rent_monthly_cost','p2_55_4':'tot_amm_loans_rest',
                       'p2_61_4' : 'tot_month_loans','p4_24':'port_val_no_pub','p4_25':'ann_income_div_no_pub',
                       'p4_15':'port_val_pub','p4_10':'has_pub_stocks','p4_27':'has_inv_funds','p4_18':'has_no_pub_stocks',
                       'p5_1':'has_retirment_plan','p2_32':'has_oth_real_state','p4_33':'has_fix_rent_inv','p4_39':'has_oth_inv',
                       'p2_1':'possesion_main_house','p2_8':'has_loans_for_main_house','p4_3':'has_saving_acc','p4_5':'has_pay_acc',
                       'p4_16':'ann_income_div_pub', 'p4_28a':'port_val_inv_funds','p4_35':'port_val_fix_rent',
                       'p4_40':'ann_income_oth_inv','p4_7_1':'tot_amm_savings_acc','p4_7_3':'tot_amm_pay_acc',
                       'p9_1':'avg_month_expenses_goods',
                       'p9_6':'future_expenses','p9_11':'risk aversion','p9_15a':'% spend future win'})

    df = df.drop(columns = columns_retirement_plans)
    
    #Fill has_loans_for_main_house as it has many NaN values
    df.loc[df['loans_tot_rest'] > 0, 'has_loans_for_main_house'] = 1
    df.loc[df['loans_tot_rest'] <= 0, 'has_loans_for_main_house'] = 2
    
    df.loc[df['tot_amm_year_retirement_plans'].isna(), 'tot_amm_year_retirement_plans'] = 0
    df.loc[df['tot_value_retirement_plans'].isna(), 'tot_value_retirement_plans'] = 0

    #Total loans rest
    df['tot_loans_rest'] = df[['tot_amm_loans_rest','loans_tot_rest','other_loans_tot_rest']].sum(axis = 1)
    df = df.drop(columns=['tot_amm_loans_rest','loans_tot_rest','other_loans_tot_rest'])

    #Total loans month
    df['tot_loans_month'] = df[['tot_month_loans','other_loans_tot_monthly','loans_tot_monthly']].sum(axis = 1)
    df = df.drop(columns=['tot_month_loans','other_loans_tot_monthly','loans_tot_monthly'])
    
    #Education:
    '''
    1 Analfabetos
    2 Primaria
    3 Enseñanza para la inserción laboral sin ESO
    4 ESO
    5 Enseñanza para la inserción laboral que precisa de ESO
    6 Bachillerato
    7 Enseñanza para la inserción laboral que precisa Bachillerato
    8 Grado superior de Formación Profesional
    9 Enseñanza de de 2 o más años que precisa bachiller
    1001 Diplomados universitarios, ingenieros técnicos, arquitectos técnicos y equivalentes
    1002 Grados universitarios, licenciados, ingenieros superiores, arquitectos y equivalentes
    11 Master
    12 Doctorado
    97 Otro
    '''
    df.loc[df['education'] == 12, 'education'] = 13
    df.loc[df['education'] == 11, 'education'] = 12
    df.loc[df['education'] == 1002, 'education'] = 11
    df.loc[df['education'] == 1001, 'education'] = 10
    df.loc[df['education'] == 8, 'education'] = 9
    df.loc[df['education'] == 7, 'education'] = 8
    df.loc[df['education'] == 6, 'education'] = 7
    df.loc[df['education'] == 97, 'education'] = 6
    #df.loc[df['education'] == 97, 'education'] = int(df['education'].mean())
    
    columns_to_drop = ['has_pub_stocks','has_inv_funds','has_no_pub_stocks','has_retirment_plan','has_oth_real_state',
                   'has_fix_rent_inv','has_oth_inv','has_loans_for_main_house','has_saving_acc','has_pay_acc',
                   'possesion_main_house','avg_loans_int','avg_time_loans_rest','other_loans_avg_time_rest','avg_age_retirement_plans']
    
    df = df.drop(columns = columns_to_drop)
    
    df['tot_amm_acc'] = df[['tot_amm_savings_acc','tot_amm_pay_acc']].sum(axis = 1)
    df = df.drop(columns = ['tot_amm_savings_acc','tot_amm_pay_acc'])
    df['port_inv_val'] = df[['port_val_no_pub','port_val_fix_rent','port_val_pub','port_val_inv_funds']].sum(axis = 1)
    df = df.drop(columns = ['port_val_no_pub','port_val_fix_rent','port_val_pub','port_val_inv_funds'])
    df['ann_inv_income'] = df[['ann_income_div_no_pub','ann_income_oth_inv','ann_income_div_pub']].sum(axis = 1)
    df = df.drop(columns = ['ann_income_div_no_pub','ann_income_oth_inv','ann_income_div_pub'])
    df['monthly_expenses'] =  df[['rent_monthly_cost','tot_loans_month','avg_month_expenses_goods']].sum(axis = 1)
    df = df.drop(columns = ['rent_monthly_cost','tot_loans_month','avg_month_expenses_goods'])
    
    df = df.drop(columns = ['h_2017','h_2014','pan_1'])
    
    df[['tot_loans_rest','monthly_expenses']] = df[['tot_loans_rest','monthly_expenses']].apply(lambda x: x * (-1))
    
    
    return df


In [281]:
df_2017 = pipeline(url_dataset_2017,url_dataset_seccion6_2017,'h_2017','h_2014')
df_2017

,age,married,education,value_main_house,future_expenses,risk aversion,% spend future win,tot_amm_year_retirement_plans,tot_value_retirement_plans,tot_income_month,tot_loans_rest,tot_amm_acc,port_inv_val,ann_inv_income,monthly_expenses
0,73,6,2,150000.0,3,4,0,0.0,0.0,637.0,-0.0,0.0,0.0,0.0,-487.0
1,74,2,4,300000.0,3,4,0,0.0,0.0,1076.0,-0.0,30000.0,0.0,0.0,-3000.0
2,74,6,4,300000.0,3,4,99,0.0,0.0,1000.0,-0.0,5000.0,0.0,0.0,-600.0
3,65,2,11,124339.0,2,4,0,0.0,0.0,2012.0,-0.0,400.0,10000.0,250.0,-1700.0
4,76,2,2,311529.0,3,4,100,17500.0,150000.0,16667.0,-0.0,1000000.0,2000000.0,500.0,-10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6408,45,4,11,84000.0,3,3,70,250.0,1701.0,2300.0,-31539.0,3000.0,0.0,0.0,-1222.0
6409,49,2,11,1000000.0,1,4,100,8000.0,90000.0,7422.0,-0.0,70000.0,600000.0,4000.0,-3000.0
6410,47,2,11,600000.0,3,3,0,5300.0,40000.0,41667.0,-250000.0,100000.0,800.0,0.0,-4500.0
6411,73,2,10,150000.0,3,4,50,0.0,0.0,1840.0,-8000.0,4200.0,4.0,0.0,-1014.0


In [282]:
from scipy import stats
import numpy as np

columns = ['value_main_house','tot_amm_year_retirement_plans','tot_value_retirement_plans','tot_income_month','tot_loans_rest','tot_amm_acc','port_inv_val','ann_inv_income','monthly_expenses']

z = np.abs(stats.zscore(df_2017[columns]))
threshold = 3
outliers = np.where(z > 3)
outliers = np.unique(outliers[0])
len(outliers)

427

In [284]:
df_2017.iloc[outliers]

,age,married,education,value_main_house,future_expenses,risk aversion,% spend future win,tot_amm_year_retirement_plans,tot_value_retirement_plans,tot_income_month,tot_loans_rest,tot_amm_acc,port_inv_val,ann_inv_income,monthly_expenses
4,76,2,2,311529.0,3,4,100,17500.0,150000.00,16667.0,-0.0,1000000.0,2000000.0,500.0,-10000.0
19,58,2,11,600000.0,3,3,50,3000.0,200000.00,0.0,-0.0,50000.0,200000.0,2000.0,-2200.0
35,50,2,7,400000.0,3,3,100,7200.0,12000.00,0.0,-40000.0,15000.0,1000.0,0.0,-4100.0
44,57,2,11,1400000.0,3,3,15,2500.0,0.00,0.0,-968000.0,5000.0,701000.0,10000.0,-7400.0
52,73,2,12,500000.0,3,4,50,8000.0,180000.00,3900.0,-170000.0,40000.0,230000.0,2500.0,-3300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6392,53,2,7,1000000.0,2,3,50,13000.0,25000.00,0.0,-0.0,50000.0,2500000.0,10000.0,-9600.0
6396,49,5,12,300000.0,2,4,0,8000.0,22258.75,5000.0,-4000.0,207500.0,209000.0,0.0,-2730.0
6397,69,2,10,1500000.0,3,4,100,0.0,0.00,11000.0,-0.0,700000.0,0.0,0.0,-6000.0
6409,49,2,11,1000000.0,1,4,100,8000.0,90000.00,7422.0,-0.0,70000.0,600000.0,4000.0,-3000.0


,age,married,education,value_main_house,future_expenses,risk aversion,% spend future win,tot_amm_year_retirement_plans,tot_value_retirement_plans,tot_income_month,tot_loans_rest,tot_amm_acc,port_inv_val,ann_inv_income,monthly_expenses
0,0.884078,2.172869,-1.329577,-0.251022,0.953768,0.509275,-1.093357,-0.244870,-0.266015,-0.367359,0.198161,-0.173417,-0.060779,-0.040646,0.704288
1,0.951329,-0.388287,-0.789578,0.140706,0.953768,0.509275,-1.093357,-0.244870,-0.266015,-0.217803,0.198161,-0.056623,-0.060779,-0.040646,-0.593085
2,0.951329,2.172869,-0.789578,0.140706,0.953768,0.509275,1.419496,-0.244870,-0.266015,-0.243694,0.198161,-0.153951,-0.060779,-0.040646,0.645950
3,0.346071,-0.388287,1.100416,-0.318036,-0.118176,0.509275,-1.093357,-0.244870,-0.266015,0.101068,0.198161,-0.171860,-0.059344,-0.039522,0.078059
4,1.085831,-0.388287,-1.329577,0.170814,0.953768,0.509275,1.444878,3.000000,3.000000,3.000000,0.198161,3.000000,0.226242,-0.038397,-3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6408,-0.998949,0.892291,1.100416,-0.423382,0.953768,-1.293142,0.683408,-0.092464,-0.228349,0.199182,0.013783,-0.161738,-0.060779,-0.040646,0.324834
6409,-0.729945,-0.388287,1.100416,1.968770,-1.190121,0.509275,1.444878,3.000000,1.726891,1.944115,0.198161,0.099102,0.025327,-0.022650,-0.593085
6410,-0.864447,-0.388287,1.100416,0.924162,0.953768,-1.293142,-1.093357,2.986155,0.619721,3.000000,-1.263345,0.215896,-0.060664,-0.040646,-1.367482
6411,0.884078,-0.388287,0.830417,-0.251022,0.953768,0.509275,0.175761,-0.244870,-0.266015,0.042472,0.151392,-0.157066,-0.060778,-0.040646,0.432217


In [289]:
#df_2017.drop(outliers, axis = 0, inplace=True)
df_2017_zscore.describe()

,age,married,education,value_main_house,future_expenses,risk aversion,% spend future win,tot_amm_year_retirement_plans,tot_value_retirement_plans,tot_income_month,tot_loans_rest,tot_amm_acc,port_inv_val,ann_inv_income,monthly_expenses
count,6.413000e+03,6.413000e+03,6.413000e+03,6413.000000,6.413000e+03,6.413000e+03,6.413000e+03,6413.000000,6413.000000,6413.000000,6413.000000,6413.000000,6413.000000,6413.000000,6413.000000
mean,5.290568e-17,2.505230e-16,9.123461e-17,-0.039583,-1.538351e-16,-1.603098e-16,4.819680e-17,-0.044085,-0.049357,-0.035621,0.036195,-0.031039,-0.025444,-0.022537,0.040027
std,1.000078e+00,1.000078e+00,1.000078e+00,0.727117,1.000078e+00,1.000078e+00,1.000078e+00,0.629706,0.620996,0.568592,0.421811,0.364625,0.200399,0.136032,0.725837
min,-2.747475e+00,-1.028576e+00,-1.599576e+00,-0.642750,-1.190121e+00,-4.897974e+00,-1.093357e+00,-0.244870,-0.266015,-0.584369,-3.000000,-0.173417,-0.060779,-0.040646,-3.000000
25%,-7.299454e-01,-3.882868e-01,-1.059578e+00,-0.486059,-1.190121e+00,5.092747e-01,-1.093357e+00,-0.244870,-0.266015,-0.392228,0.108611,-0.167577,-0.060779,-0.040646,-0.128447
50%,9.815520e-03,-3.882868e-01,2.041940e-02,-0.251022,-1.181763e-01,5.092747e-01,1.757608e-01,-0.244870,-0.266015,-0.141492,0.198161,-0.142272,-0.060779,-0.040646,0.253589
75%,8.168275e-01,-3.882868e-01,1.100416e+00,0.140706,9.537684e-01,5.092747e-01,1.444878e+00,-0.244870,-0.243207,0.196797,0.198161,-0.056623,-0.059057,-0.040646,0.491071
max,1.691090e+00,2.172869e+00,1.640415e+00,3.000000,9.537684e-01,5.092747e-01,1.444878e+00,3.000000,3.000000,3.000000,0.198161,3.000000,3.000000,3.000000,0.922668


In [257]:
df_2017.describe()

,age,married,education,value_main_house,future_expenses,risk aversion,% spend future win,tot_amm_year_retirement_plans,tot_value_retirement_plans,tot_income_month,tot_loans_rest,tot_amm_acc,port_inv_val,ann_inv_income,monthly_expenses
count,5986.000000,5986.000000,5986.000000,5.986000e+03,5986.000000,5986.000000,5986.000000,5986.000000,5986.000000,5986.000000,5986.000000,5986.000000,5.986000e+03,5986.000000,5986.000000
mean,59.511193,2.618109,6.689609,1.941933e+05,2.095222,3.745406,43.069161,189.762332,5483.801652,1459.379218,-23133.645673,26692.639554,1.239728e+05,2083.223187,-1580.177092
std,15.044161,1.579262,3.671637,2.140624e+05,0.938522,0.533584,39.326114,643.912165,16784.142255,1338.460213,54634.495841,58814.275614,6.579773e+05,14645.264077,1070.448357
min,19.000000,1.000000,1.000000,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-530000.000000,0.000000,0.000000e+00,0.000000,-7624.000000
25%,48.000000,2.000000,3.000000,5.996775e+04,1.000000,4.000000,0.000000,0.000000,0.000000,528.250000,-15000.000000,1300.000000,0.000000e+00,0.000000,-2000.000000
50%,60.000000,2.000000,7.000000,1.328885e+05,2.000000,4.000000,50.000000,0.000000,0.000000,1220.000000,-0.000000,7000.000000,0.000000e+00,0.000000,-1300.000000
75%,72.000000,2.000000,10.000000,2.500000e+05,3.000000,4.000000,90.000000,0.000000,0.000000,2190.000000,-0.000000,25000.000000,3.974000e+03,0.000000,-870.000000
max,85.000000,6.000000,13.000000,1.368927e+06,3.000000,4.000000,100.000000,5100.000000,147000.000000,10400.000000,-0.000000,800000.000000,1.640000e+07,638000.000000,-64.000000


In [258]:
os.makedirs('final_dataset', exist_ok=True)  
df_2017.to_csv('final_dataset/df_2017.csv', index = False) 

In [291]:
df_2017_red = df_2017.copy()

df_2017_red.loc[df_2017_red['married'] == 3, 'married'] = 2 #POLITICAL = MARRIED
df_2017_red.loc[df_2017_red['married'] == 4, 'married'] = 3 #SEPARATED
df_2017_red.loc[df_2017_red['married'] == 5, 'married'] = 3 #DIVORCED = SEPARATED
df_2017_red.loc[df_2017_red['married'] == 6, 'married'] = 1 #WIDOWS = SINGLE

df_2017_red.loc[df_2017_red['education'] == 3, 'education'] = 2 #ELEMENTARY / WORK NO ESO
df_2017_red.loc[df_2017_red['education'] == 4, 'education'] = 3 #ESO
df_2017_red.loc[df_2017_red['education'] == 5, 'education'] = 3 #WORK ESO
df_2017_red.loc[df_2017_red['education'] == 6, 'education'] = 4 #OTHER
df_2017_red.loc[df_2017_red['education'] == 7, 'education'] = 5 #WORK BATX/BATX/FP
df_2017_red.loc[df_2017_red['education'] == 8, 'education'] = 5 #WORK BATX/BATX/FP
df_2017_red.loc[df_2017_red['education'] == 9, 'education'] = 5 #WORK BATX/BATX/FP
df_2017_red.loc[df_2017_red['education'] == 10, 'education'] = 6 #UNI
df_2017_red.loc[df_2017_red['education'] == 11, 'education'] = 6 #UNI
df_2017_red.loc[df_2017_red['education'] == 12, 'education'] = 7 #MASTER
df_2017_red.loc[df_2017_red['education'] == 13, 'education'] = 8 #PHD

os.makedirs('final_dataset', exist_ok=True)  
df_2017_red.to_csv('final_dataset/df_2017_red.csv', index = False) 

In [292]:
df_2017_red_2 = df_2017_red.copy()

df_2017_red_2.loc[df_2017_red_2['married'] == 3, 'married'] = 1 #SEPARATED/DIVORCED = SINGLE
os.makedirs('final_dataset', exist_ok=True)  
df_2017_red_2.to_csv('final_dataset/df_2017_red_2.csv', index = False) 

In [293]:
df_2017_zscore = df_2017_red_2.apply(stats.zscore)
for col in columns:
    df_2017_zscore[col].loc[df_2017_zscore[col] > 3] = 3
    df_2017_zscore[col].loc[df_2017_zscore[col] < -3] = -3
    
df_2017_zscore

,age,married,education,value_main_house,future_expenses,risk aversion,% spend future win,tot_amm_year_retirement_plans,tot_value_retirement_plans,tot_income_month,tot_loans_rest,tot_amm_acc,port_inv_val,ann_inv_income,monthly_expenses
0,0.884078,-1.304794,-1.315977,-0.251022,0.953768,0.509275,-1.093357,-0.244870,-0.266015,-0.367359,0.198161,-0.173417,-0.060779,-0.040646,0.704288
1,0.951329,0.766405,-0.762104,0.140706,0.953768,0.509275,-1.093357,-0.244870,-0.266015,-0.217803,0.198161,-0.056623,-0.060779,-0.040646,-0.593085
2,0.951329,-1.304794,-0.762104,0.140706,0.953768,0.509275,1.419496,-0.244870,-0.266015,-0.243694,0.198161,-0.153951,-0.060779,-0.040646,0.645950
3,0.346071,0.766405,0.899514,-0.318036,-0.118176,0.509275,-1.093357,-0.244870,-0.266015,0.101068,0.198161,-0.171860,-0.059344,-0.039522,0.078059
4,1.085831,0.766405,-1.315977,0.170814,0.953768,0.509275,1.444878,3.000000,3.000000,3.000000,0.198161,3.000000,0.226242,-0.038397,-3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6408,-0.998949,-1.304794,0.899514,-0.423382,0.953768,-1.293142,0.683408,-0.092464,-0.228349,0.199182,0.013783,-0.161738,-0.060779,-0.040646,0.324834
6409,-0.729945,0.766405,0.899514,1.968770,-1.190121,0.509275,1.444878,3.000000,1.726891,1.944115,0.198161,0.099102,0.025327,-0.022650,-0.593085
6410,-0.864447,0.766405,0.899514,0.924162,0.953768,-1.293142,-1.093357,2.986155,0.619721,3.000000,-1.263345,0.215896,-0.060664,-0.040646,-1.367482
6411,0.884078,0.766405,0.899514,-0.251022,0.953768,0.509275,0.175761,-0.244870,-0.266015,0.042472,0.151392,-0.157066,-0.060778,-0.040646,0.432217


In [294]:
os.makedirs('final_dataset', exist_ok=True)  
df_2017_zscore.to_csv('final_dataset/df_2017_zscore.csv', index = False) 

In [261]:
def categorize(dataframe, name):
    married_cols = ['SINGLE', 'MARRIED', 'POLITICAL', 'SEPARATED', 'DIVORCED', 'WIDOW']

    for i in range(0,len(married_cols)):
        dataframe.loc[dataframe['married'] == i + 1, 'married'] = married_cols[i]

    edu_cols = ['ANALPHABET','ELEMENTARY','WORK NO ESO','ESO','WORK AND ESO','OTHER','BATX','WORK AND BATX','FP','DIP UNI','DEG UNI','MASTER','PHD']

    for i in range(0,len(edu_cols)):
        dataframe.loc[dataframe['education'] == i + 1, 'education'] = edu_cols[i]
        
    os.makedirs('final_dataset', exist_ok=True)  
    dataframe.to_csv('final_dataset/{n}'.format(n = name), index = False)

In [262]:
df_2017_cat = df_2017.copy()
categorize(df_2017_cat,'df_2017_cat.csv')

df_2017_red_cat = df_2017_red.copy()
categorize(df_2017_red_cat,'df_2017_red_cat.csv')

df_2017_red_cat_2 = df_2017_red_2.copy()
categorize(df_2017_red_cat_2,'df_2017_red_cat_2.csv')
 